クロスデバイス　クロスリスティングとの併用

In [1]:
# ライブラリimport
import argparse
import logging
import os

import pandas as pd
from pandas import DataFrame
import pandas.tseries.offsets as offsets
import numpy as np
from datetime import date, timedelta, time, datetime
from tabulate import tabulate

from itertools import product

from ailab_tools.smn import ImpalaResource
from ailab_tools.utils import GMail, GSheet

from collections import OrderedDict

import zipfile
import glob

import gc

from ailab_tools.magics import slack_notify

2630	(unico)株式会社ミサワ

In [2]:
today_yymmdd = date.today().strftime("%y%m%d")
print(today_yymmdd)

190522


In [3]:
#デバイス
seed_sp_device = 'iOS','Android'
xd_sp_device = 'iphone','android_phone','windows_phone'

#ターゲット
advertiser_id = 2630

セグメントIDを確認する

In [4]:
#f-string
query = (
    f"""
    select *
    from 
    dm.segment_hierarchies
    where 
    segment_name like "00000000-90days-UNICO"
    and dmp_id = 10050
    """
    )

with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
        df_seg = ir.sql_to_pandas(query)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
    s

In [5]:
df_seg.head()

,dmp_id,segment_owner_id,org_segment_owner_id,segment_owner_name,inhouse_only,owner_status,segment_id,org_segment_id,segment_name,segment_status,logic,negative,weight,target_segment_logic_id,target_id
0,10050,25,00000000,00000000,0,1,1904,25e19ca116d1907,00000000-90days-UNICO,1,0,0,0,174192,282431
1,10050,25,00000000,00000000,0,1,1904,25e19ca116d1907,00000000-90days-UNICO,1,0,0,0,174193,282432


In [6]:
segment_id = "25e19ca116d1907"
print(segment_id)

25e19ca116d1907


In [7]:
seed_table = f"dm_tmp.ya_seed_advid_{advertiser_id}_segid_{segment_id}_{today_yymmdd}"
print(seed_table)

dm_tmp.ya_seed_advid_2630_segid_25e19ca116d1907_190522


In [8]:
uid_table = f"dm_tmp.ya_xd_advid_{advertiser_id}_listing_segid_{segment_id}_{today_yymmdd}"
print(uid_table)

dm_tmp.ya_xd_advid_2630_listing_segid_25e19ca116d1907_190522


In [9]:
#デバイス
seed_sp_device = 'iOS','Android'
xd_sp_device = 'iphone','android_phone','windows_phone','android_tablet','ipad','windows_tablet'

In [10]:
%%slack_notify @yusuke_akada

#f-string
query = (
    f"""
    select 
    distinct 
    db2.smn_uid
    /*,case when db2.device in ('iphone','android_phone','windows_phone') then 'SP' ELSE 'PC' END as xd_device*/
    from segment.drawbridge_device db
    inner join dm.dmp_seg_user_list seg
    on db.smn_uid = seg.smn_uid
    inner join segment.drawbridge_device db2
    on db.drawbridge_consumer_id = db2.drawbridge_consumer_id
    where concat_ws('-',year,month,day) between to_date(days_add(now(),-30)) and to_date(days_add(now(),-1))
    and org_segment_id in ('{segment_id}')
    and dmp_id = 10050
    and db2.smn_uid > 0
    and seg.smn_uid <> db2.smn_uid
    """
    )

with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
        df = ir.sql_to_pandas(query)
        ir.upload_dataframe(uid_table, df, ['bigint'])

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
    s

In [11]:
df.shape[0]

1024851